In [25]:

#######################        BOARD CLASS        ###########################
# The Board class is the data structure that holds the Connect 4 boards and the game operations

# The Connect 4 board is 7 cells wide and 6 cells tall

# The underlying data structure is a 2-d list
# The first dimension is the column; the second dimension is the row

# Every cell in the above list contains either a 0 or a 1. Player 1 is represented by 0 tiles, and Player

##############################################################################
class Board(object):

    #static class variables - shared across all instances
    HEIGHT = 6
    WIDTH = 7

    def __init__(self, orig=None):
        self.legal_players = [0,1]
        self.board = [[] for x in range(self.WIDTH)]
        self.numMoves = 0
        self.lastMove = None
        return

    def copy(self):
        new_board = self.__class__()
        new_board.board = [row[:] for row in self.board]
        new_board.numMoves = self.numMoves
        return new_board
    ########################################################################
    #                           Mutations
    ########################################################################

    # Puts a piece in the appropriate column and checks to see if it was a winning move
    # Pieces are either 1 or 0; automatically decided
    def move(self, column, turn):
        # update board data
        self.numMoves += 1
        self.board[column].append(turn)

    def get_moves(self):
        return [i for i in range(self.WIDTH) if len(self.board[i])<self.HEIGHT]
    ########################################################################
    #                           Observations
    ########################################################################

    # Generates a list of the valid children of the board
    # A child is of the form (move_to_make_child, child_object)
    def children(self):
        children = []
        for i in range(7):
            if len(self.board[i]) < 6:
                child = Board(self)
                child.makeMove(i)
                children.append((i, child))
        return children

    # Returns:
    #  -1 if game is not over
    #   0 if the game is a draw
    #   1 if player 1 wins
    #   2 if player 2 wins
    def isTerminal(self):
        for i in range(0,self.WIDTH):
            for j in range(0,self.HEIGHT):
                try:
                    if self.board[i][j]  == self.board[i+1][j] == self.board[i+2][j] == self.board[i+3][j]:
                        return self.board[i][j] + 1
                except IndexError:
                    pass

                try:
                    if self.board[i][j]  == self.board[i][j+1] == self.board[i][j+2] == self.board[i][j+3]:
                        return self.board[i][j] + 1
                except IndexError:
                    pass

                try:
                    if not j + 3 > self.HEIGHT and self.board[i][j] == self.board[i+1][j + 1] == self.board[i+2][j + 2] == self.board[i+3][j + 3]:
                        return self.board[i][j] + 1
                except IndexError:
                    pass

                try:
                    if not j - 3 < 0 and self.board[i][j] == self.board[i+1][j - 1] == self.board[i+2][j - 2] == self.board[i+3][j - 3]:
                        return self.board[i][j] + 1
                except IndexError:
                    pass
        if self.numMoves == 42:
            return 0
        return -1
 ########################################################################
    #                           Utilities
    ########################################################################

    # Prints out a visual representation of the board
    # X's are 1's and 0's are 0s
    def print(self):
        print("")
        print("+" + "---+" * self.WIDTH)
        for rowNum in range(self.HEIGHT - 1, -1, -1):
            row = "|"
            for colNum in range(self.WIDTH):
                if len(self.board[colNum]) > rowNum:
                    row += " " + ('X' if self.board[colNum][rowNum] else 'O') + " |"
                else:
                    row += "   |"
            print(row)
            print("+" + "---+" * self.WIDTH)
        try:
            print(self.lastMove[1])
        except:
            pass
        print(self.numMoves)

    def score_game(self):
        heur = 0
        state = self.board
        for i in range(0, self.WIDTH):
            for j in range(0, self.HEIGHT):
                # check horizontal streaks
                try:
                    # add player one streak scores to heur
                    if state[i][j] == state[i + 1][j] == 0:
                        heur += 10
                    if state[i][j] == state[i + 1][j] == state[i + 2][j] == 0:
                        heur += 100
                    if state[i][j] == state[i+1][j] == state[i+2][j] == state[i+3][j] == 0:
                        heur += 10000

                    # subtract player two streak score to heur
                    if state[i][j] == state[i + 1][j] == 1:
                        heur -= 10
                    if state[i][j] == state[i + 1][j] == state[i + 2][j] == 1:
                        heur -= 100
                    if state[i][j] == state[i+1][j] == state[i+2][j] == state[i+3][j] == 1:
                        heur -= 10000
                except IndexError:
                    pass

                # check vertical streaks
                try:
                    # add player one vertical streaks to heur
                    if state[i][j] == state[i][j + 1] == 0:
                        heur += 10
                    if state[i][j] == state[i][j + 1] == state[i][j + 2] == 0:
                        heur += 100
                    if state[i][j] == state[i][j+1] == state[i][j+2] == state[i][j+3] == 0:
                        heur += 10000

                    # subtract player two streaks from heur
                    if state[i][j] == state[i][j + 1] == 1:
                        heur -= 10
                    if state[i][j] == state[i][j + 1] == state[i][j + 2] == 1:
                        heur -= 100
                    if state[i][j] == state[i][j+1] == state[i][j+2] == state[i][j+3] == 1:
                        heur -= 10000
                except IndexError:
                    pass

                # check positive diagonal streaks
                try:
                    # add player one streaks to heur
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i + 1][j + 1] == 0:
                        heur += 100
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i + 1][j + 1] == state[i + 2][j + 2] == 0:
                        heur += 100
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i+1][j + 1] == state[i+2][j + 2] \
                            == state[i+3][j + 3] == 0:
                        heur += 10000

                    # add player two streaks to heur
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i + 1][j + 1] == 1:
                        heur -= 100
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i + 1][j + 1] == state[i + 2][j + 2] == 1:
                        heur -= 100
                    if not j + 3 > self.HEIGHT and state[i][j] == state[i+1][j + 1] == state[i+2][j + 2] \
                            == state[i+3][j + 3] == 1:
                        heur -= 10000
                except IndexError:
                    pass

                # check negative diagonal streaks
                try:
                    # add  player one streaks
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == 0:
                        heur += 10
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == state[i+2][j - 2] == 0:
                        heur += 100
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == state[i+2][j - 2] \
                            == state[i+3][j - 3] == 0:
                        heur += 10000

                    # subtract player two streaks
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == 1:
                        heur -= 10
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == state[i+2][j - 2] == 1:
                        heur -= 100
                    if not j - 3 < 0 and state[i][j] == state[i+1][j - 1] == state[i+2][j - 2] \
                            == state[i+3][j - 3] == 1:
                        heur -= 10000
                except IndexError:
                    pass
        return heur


import math

class PlayerMM:
    def __init__(self):
        self.bestmove = {}

    def _mini_max(self, game, player, depth):
        if player not in self.bestmove:
            self.bestmove[player] = {}
            
        player_memo = self.bestmove[player]
        
        if True:
            if game.isTerminal()>-1:
                best_move = None
                if game.isTerminal == 0:
                    best_score = 0
                else:
                    if game.isTerminal == player+1:
                        best_score = math.inf
                    else:
                        best_score = -math.inf
            elif depth != 0:
                alt_player = 1 - player
                moves = game.get_moves() 
                best_score = -math.inf
                for move in moves: 
                    clone = game.copy()
                    clone.move(move, player)
                    score = self._mini_max(clone, player=alt_player,depth = depth-1)[1]
                    score *= -1 
                    if score >= best_score:
                        best_move = move
                        best_score = score 
            else:
                best_move = None
                best_score = game.score_game()
                if player == 1:
                    best_score *= -1
            try:
                self.bestmove[player][game] = (best_move, best_score)
            except:
                game.print()
                print(game.isTerminal())
                print(game.numMoves)
        try:
            return self.bestmove[player][game]
        except:
            return (0,0)


"""class PlayerAB(Player):

    def __init__(self, depthLimit, isPlayerOne):
        super().__init__(depthLimit, isPlayerOne)

    # returns the optimal column to move in by implementing the Alpha-Beta algorithm
    def findMove(self, board):
        score, move = self.alphaBeta(board, self.depthLimit, self.isPlayerOne, -math.inf, math.inf)
        return move

    # findMove helper function, utilizing alpha-beta pruning within the  minimax algorithm
    def alphaBeta(self, board, depth, player, alpha, beta):
        if board.isTerminal() == 0:
            return -math.inf if player else math.inf, -1
        elif depth == 0:
            return self.heuristic(board), -1

        if player:
            bestScore = -math.inf
            shouldReplace = lambda x: x > bestScore
        else:
            bestScore = math.inf
            shouldReplace = lambda x: x < bestScore

        bestMove = -1

        children = board.children()
        for child in children:
            move, childboard = child
            temp = self.alphaBeta(childboard, depth-1, not player, alpha, beta)[0]
            if shouldReplace(temp):
                bestScore = temp
                bestMove = move
            if player:
                alpha = max(alpha, temp)
            else:
                beta = min(beta, temp)
            if alpha >= beta:
                break
        return bestScore, bestMove


class ManualPlayer(Player):
    def findMove(self, board):
        opts = " "
        for c in range(board.WIDTH):
            opts += " " + (str(c + 1) if len(board.board[c]) < 6 else ' ') + "  "
        print(opts)

        col = input("Place an " + ('O' if self.isPlayerOne else 'X') + " in column: ")
        col = int(col) - 1
        return col

class Game:


    def __init__(self, startBoard, player1, player2):
        self.startBoard = startBoard
        self.player1 = player1
        self.player2 = player2

    ########################################################################
    #                     Simulate a Local Game
    ########################################################################

    def simulateLocalGame(self):

        board = Board(orig=self.startBoard)
        isPlayer1 = True

        while(True):

            #finds the move to make
            if isPlayer1:
                move = self.player1.findMove(board)
            else:
                move = self.player2.findMove(board)

            #makes the move
            board.makeMove(move)
            board.print()

            #determines if the game is over or not
            isOver = board.isTerminal()
            if isOver == 0:
                print("It is a draw!")
                break
            elif isOver == 1:
                print("Player 1 wins!")
                break
            elif isOver == 2:
                print("Player 2 wins!")
                break
            else:
                isPlayer1 = not isPlayer1


"""

game = Board()
bot = PlayerMM()
"""
game.move(5)
game.move(1)
game.print()
game2 = game.copy()
game2.move(3)
game.print()
game2.print()"""
#movaghat
"""
game.move(1,0)
game.move(1,0)
game.move(1,1)
game.move(1,0)
game.move(1,1)
game.move(1,0)
game.move(0,1)
game.move(0,1)
game.move(0,0)
game.move(2,0)
game.move(2,1)
game.move(2,1)
game.move(2,0)
game.move(3,0)
game.move(3,0)
game.move(3,0)
game.move(3,1)
game.move(3,0)
game.move(3,0)
game.move(4,1)
game.move(4,0)
game.move(4,1)
game.move(4,1)
game.move(4,1)
game.move(4,0)
game.move(5,1)
game.move(5,0)
game.move(5,1)
game.move(5,1)
game.move(5,1)
game.move(5,0)
game.move(6,1)
game.move(6,1)
game.move(6,0)
game.move(6,0)
game.move(6,0)
game.move(6,1)
game.print()
#movaghat"""
human = None
while human is None:
    human = int(input("Select player: {} ".format(game.legal_players)).upper())
    if human not in game.legal_players:
        print("Invalid option")
        human = None

comp = [alt_player for alt_player in game.legal_players if alt_player != human][0]
turn = 0
while game.isTerminal()==-1: 
    if comp == turn:
        move = bot._mini_max(game, comp, 6)[0]
    else:
        move = bot._mini_max(game, 1- comp, 6)[0]
    if move in game.get_moves():
        game.move(move, turn)
        turn = comp if turn == human else human
    else:
        print("Illegal move.")
    game.print()
game.print()


+---+---+---+---+---+---+---+
|   | O |   | O | O | O | X |
+---+---+---+---+---+---+---+
|   | X |   | O | X | X | O |
+---+---+---+---+---+---+---+
|   | O | O | X | X | X | O |
+---+---+---+---+---+---+---+
| O | X | X | O | X | X | O |
+---+---+---+---+---+---+---+
| X | O | X | O | O | O | X |
+---+---+---+---+---+---+---+
| X | O | O | O | X | X | X |
+---+---+---+---+---+---+---+
37
Select player: [0, 1] 0

+---+---+---+---+---+---+---+
|   | O |   | O | O | O | X |
+---+---+---+---+---+---+---+
|   | X | X | O | X | X | O |
+---+---+---+---+---+---+---+
|   | O | O | X | X | X | O |
+---+---+---+---+---+---+---+
| O | X | X | O | X | X | O |
+---+---+---+---+---+---+---+
| X | O | X | O | O | O | X |
+---+---+---+---+---+---+---+
| X | O | O | O | X | X | X |
+---+---+---+---+---+---+---+
38

+---+---+---+---+---+---+---+
|   | O | O | O | O | O | X |
+---+---+---+---+---+---+---+
|   | X | X | O | X | X | O |
+---+---+---+---+---+---+---+
|   | O | O | X | X | X | O |
+---+--